In [1]:
import numpy as np
import io
import os
import pandas as pd
from scipy.stats import chisquare
from scipy.stats import hypergeom
from scipy.stats import fisher_exact
import bdsg
import copy
import re 
from collections import Counter

# Tests

In [2]:
### test chi deux ###
f_exp = np.array([44, 24, 29, 3]) / 100 * 189
f_obs = np.array([43, 52, 54, 40])
print("chi deux : ", chisquare(f_obs=f_obs, f_exp=f_exp))

chi deux :  Power_divergenceResult(statistic=228.23515947653874, pvalue=3.3295585338846486e-49)


In [3]:
### test exact fisher ###
table = np.array([[6, 2], [1, 4]])
M = table.sum()
n = table[0].sum()
N = table[:, 0].sum()
start, end = hypergeom.support(M, n, N)
hypergeom.pmf(np.arange(start, end + 1), M, n, N)
np.array([0.01631702, 0.16317016, 0.40792541, 0.32634033, 0.08158508,
       0.004662])
res = fisher_exact(table, alternative='two-sided')
print("fisher : ", res.pvalue)

fisher :  0.10256410256410256


# VCF

In [4]:
# on va essayer de faire un giga tableau qui recense tous les chemins possibles -> OK (pour un éch mais normalement c'est les mêmes chemins possibles)
# une fois qu'on aura fait ce giga tableau, on va essayer de le transformer en giga matrice qui compte pour 1 éch les chemins pris -> OK
# une fois que ce sera fait on pourra transformer la giga matrice en giga table de contingence (?) qui recense la même chose mais pour tous les échantillons -> OK (dataframe)

In [5]:
# fonctions annexes pour créer le df qui contient les infos après avoir parse tous les vcf

In [45]:
# read ONE vcf file

def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

# verification de la fonction read_vcf 

vcf1_df = read_vcf("/home/yboulkaid/Documents/sample_data/pgtest.data/calls/samp_g0_0.vcf")
vcf1_df.head(n = 100)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,SAMPLE
0,ref,450,>1>4,T,G,133.927,PASS,"AT=>1>2>4,>1>3>4;DP=46",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:46:46,0:-2.378747,-15.294357,-107.194421:1..."
1,ref,578,>4>9,TTTGCGGGCCGTCAAGATGAACTGGTGCCTGTAGGATTATGTCCTC...,TTTGCGGGCCGTCAAGATGAACTGGTGCCTGTAGGATTATGTCCTC...,14.6171,PASS,"AT=>4>5>7>8>9,>4>5>6>8>9;DP=50",GT:DP:AD:GL:GQ:GP:XD:MAD,"1/0:50:25,25:-3.306431,-2.422690,-3.306431:16:..."
2,ref,1118,>9>12,A,G,177.783,PASS,"AT=>9>11>12,>9>10>12;DP=61",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:61:61,0:-2.701171,-20.002352,-141.869830:1..."
3,ref,1340,>12>14,GATCGGACTTCTTAACGGGTTCCTCACGTAGCGATCTCTACGGGAA...,G,116.981,PASS,"AT=>12>13>14,>12>14;DP=47",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:47:46,1:-2.770745,-13.991746,-90.540504:11..."
4,ref,1529,>14>19,C,CCCGATCTATGTCGAGGCTTTCGGGCAGGGCCGCTATTAACATCGT...,175.5,PASS,"AT=>14>19,>14>15>17>18>19,>14>15>16>18>19;DP=60",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:60:60,0,0:-2.835631,-19.908524,-19.908524,..."
...,...,...,...,...,...,...,...,...,...,...
95,ref,27003,>489>492,C,T,411.605,PASS,"AT=>489>490>492,>489>491>492;DP=43",GT:DP:AD:GL:GQ:GP:XD:MAD,"1/0:43:20,23:-43.061833,-2.378411,-35.907321:2..."
96,ref,27134,>492>497,A,ATACACTAGACGCCCGGGATGTACATATCGTGACTCGTTCCTAGTC...,145.789,PASS,"AT=>492>497,>492>493>495>496>497,>492>493>494>...",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:50:50,0,0:-2.439843,-16.541658,-16.541658,..."
97,ref,27275,>497>502,TGCAATAGTTTGCTCGTGGTCAGTATTTTCGGTATAAAACAGAGTT...,"T,TGCAATAGTTTGCTCGTGGTCAGTATTTTCGGTATAAAACAGAG...",371.786,PASS,"AT=>497>498>500>501>502,>497>502,>497>498>499>...",GT:DP:AD:GL:GQ:GP:XD:MAD,"1/2:52:2,24,27:-39.603567,-3.504039,-35.913027..."
98,ref,27745,>502>507,CGGACATGTAGCAGGACTCCTTATAGTTAATGTTCACTATTTAAAG...,CGGACATGTAGCAGGACTCCTTATAGTTAATGTTCACTATTTAAAG...,122.081,PASS,"AT=>502>503>504>506>507,>502>503>505>506>507,>...",GT:DP:AD:GL:GQ:GP:XD:MAD,"0/0:42:42,0,0:-2.420888,-14.151905,-14.151905,..."


In [7]:
def chemins(vcf_df):    
    chemins_possibles = []
    for i in range(len(vcf_df["INFO"])):
        text = vcf_df["INFO"][i]
        m = re.search('AT=>(.+?);DP', text)
        if m:
            found = m.group(1)
        chemins_possibles.append(found)
        
    for i in range(len(chemins_possibles)):
        chemins_possibles[i] = chemins_possibles[i].split(',')
        
    ########################################################################
    chemins_pris = []
    for i in vcf_df["SAMPLE"]:
        found = i[0:3]
        chemins_pris.append(found)
    
    for i in range(len(chemins_pris)):
        chemins_pris[i] = chemins_pris[i].split('/')
        for j in range(len(chemins_pris[i])):
            chemins_pris[i][j] = int(chemins_pris[i][j])
            
    ########################################################################
    chemins_combines = copy.deepcopy(chemins_pris)
    
    for i in range(len(chemins_pris)):
        chemins_combines[i][0] = chemins_possibles[i][chemins_pris[i][0]]
        chemins_combines[i][1] = chemins_possibles[i][chemins_pris[i][1]]
        
    ''' cas simple pour comprendre
    print(chemins_pris[0])
    chemins_pris[0][0] = chemins_possibles[0][chemins_pris[0][0]]
    chemins_pris[0][1] = chemins_possibles[0][chemins_pris[0][1]]
    print(chemins_pris[0])'''
    
    chemins_possibles = sum(chemins_possibles, [])
    chemins_pris = sum(chemins_pris, [])
    chemins_combines = sum(chemins_combines, [])
    
    return chemins_possibles, chemins_pris, chemins_combines

In [8]:
# verification de la fonction chemins

chemins_possibles1, chemins_pris1, chemins_combines1 = chemins(vcf1_df)
print('========== possibles ==========')
print(chemins_possibles1[0:5])
print('============ pris =============')
print(chemins_pris1[0:5])
print('========== combinés ===========')
print(chemins_combines1[0:5])

========== possibles ==========
['1>2>4', '>1>3>4', '4>5>7>8>9', '>4>5>6>8>9', '9>11>12']
============ pris =============
[0, 0, 1, 0, 0]
========== combinés ===========
['1>2>4', '1>2>4', '>4>5>6>8>9', '4>5>7>8>9', '9>11>12']


In [9]:
def times_taken(chemins_possibles, chemins_pris, chemins_combines, df):    
    combine_count = list((x,chemins_combines.count(x)) for x in set(chemins_combines))
    for i in range(len(combine_count)):
        for j in range(len(chemins_pris)):
            if combine_count[i][0] == str(df['snarl'][j]):
                df.loc[j, "times taken"] += combine_count[i][1]

In [10]:
#commentaires bizarres je sais pas où les placer

# on refait la même chose mais pour tous les VCF 
# création d'un df avec autant de lignes que de fichiers VCF 
# et mtn qu'on a un tout beau data frame on peut faire tout pareil mais en parsant tous les VCF :D 
# on refait pareil mais en séparant les groupes g0 et g1

In [11]:
# read all vcf files and store them in a list

all_vcf = os.listdir("/home/yboulkaid/Documents/sample_data/pgtest.data/calls/")
g0_vcf =  []
g1_vcf = []
for i in all_vcf:
    if 'g0' in i:
        g0_vcf.append(i)
    elif 'g1' in i:
        g1_vcf.append(i)

In [12]:
def create_df(which_vcf_list):
    df = pd.DataFrame(columns=['snarl index', 'snarl', 'times taken'])
    df['snarl'] = chemins_possibles1 
    df['times taken'] = 0
    
    # fill 'snarl index' column
    for i in range(len(df['snarl index'])):
        if df['snarl'][i][0] == '>':
            S = re.search('>(.+?)>', df['snarl'][i])
            if S:
                s = S.group(1)
            E = re.search('.+>(.*)', df['snarl'][i])
            if E:
                e = E.group(1)
            df.loc[i, "snarl index"] = s + '>' + e    
        else:
            S = re.search('(.+?)>', df['snarl'][i])
            if S:
                s = S.group(1)
            E = re.search('.+>(.*)', df['snarl'][i])
            if E:
                e = E.group(1)
            df.loc[i, "snarl index"] = s + '>' + e
    
            
    # fill 'times taken' column
    for i in which_vcf_list:
        vcf_df = read_vcf("/home/yboulkaid/Documents/sample_data/pgtest.data/calls/"+i)
        chemins_possibles, chemins_pris, chemins_combines = chemins(vcf_df)
        times_taken(chemins_possibles, chemins_pris, chemins_combines, df)
    
    df.set_index('snarl index', inplace=True, drop=True)
    return df

In [13]:
# complete df
snarl_df = create_df(all_vcf)
snarl_df.head(n=20)

,snarl,times taken
snarl index,,
1>4,1>2>4,51
1>4,>1>3>4,69
4>9,4>5>7>8>9,53
4>9,>4>5>6>8>9,59
9>12,9>11>12,110
9>12,>9>10>12,10
12>14,12>13>14,78
12>14,>12>14,42
14>19,14>19,64


In [14]:
# group0 df
g0_df = create_df(g0_vcf)
g0_df.head(n=20)

,snarl,times taken
snarl index,,
1>4,1>2>4,27
1>4,>1>3>4,33
4>9,4>5>7>8>9,23
4>9,>4>5>6>8>9,32
9>12,9>11>12,54
9>12,>9>10>12,6
12>14,12>13>14,43
12>14,>12>14,17
14>19,14>19,36


In [15]:
# group1 df
g1_df = create_df(g1_vcf)
g1_df.head(n=25)

,snarl,times taken
snarl index,,
1>4,1>2>4,24
1>4,>1>3>4,36
4>9,4>5>7>8>9,30
4>9,>4>5>6>8>9,27
9>12,9>11>12,56
9>12,>9>10>12,4
12>14,12>13>14,35
12>14,>12>14,25
14>19,14>19,28


In [41]:
# mtn on utilise g0_df et g1_df pour créer des tables de contingences (en précisant pour quel snarl on veut créer la table)

def table_contingence(which_snarl):
    n = snarl_df.index.value_counts()[which_snarl]
    '''if n == 2:
        thing1 = g0_df.loc[which_snarl]['snarl'][0]
        thing2 = g0_df.loc[which_snarl]['snarl'][1]
        
        df2 = pd.DataFrame(columns=['g0', 'g1'], index=[thing1, thing2])
        
        df2.at[thing1, 'g0'] = g0_df.loc[which_snarl]['times taken'][0]
        df2.at[thing2, 'g0'] = g0_df.loc[which_snarl]['times taken'][1]
        df2.at[thing1, 'g1'] = g1_df.loc[which_snarl]['times taken'][0]
        df2.at[thing2, 'g1'] = g1_df.loc[which_snarl]['times taken'][1]
    else:'''
    thing = []
    for i in range(n):
        thing.append(g0_df.loc[which_snarl]['snarl'].iloc[i])
        #thing.append(g0_df.loc[which_snarl]['snarl'][i])
        # corpus_df.loc['it'][1]
    df2 = pd.DataFrame(columns=['g0', 'g1'], index=thing)
    for i in range(len(thing)):
        df2.at[thing[i], 'g1'] = g1_df.loc[which_snarl]['times taken'].iloc[i]
        df2.at[thing[i], 'g0'] = g0_df.loc[which_snarl]['times taken'].iloc[i]
    return df2


In [42]:
table_contingence('1>4')

,g0,g1
1>2>4,27,24
>1>3>4,33,36


In [43]:
# et là on teste (test de chi deux)
df_test1 = table_contingence('1>4')
df_test2 = table_contingence('14>19')

def chi2(table):
    print("p-valeur = ", chisquare(table['g0'], table['g1']).pvalue)

chi2(df_test1)
chi2(df_test2)

p-valeur =  0.42919530044034926
p-valeur =  0.027203881751899518


In [40]:
for i in snarl_df.index.unique():
    table = table_contingence(snarl_df, i)
    chi2(table_contingence(snarl_df, i))

p-valeur =  0.42919530044034926


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.03636363636363636

In [23]:

type(snarl_df.index)

pandas.core.indexes.base.Index

 # 'Play' with bdsg
##### am i playing with bdsg or is bdsg playing with me

In [ ]:
from bdsg.bdsg import HashGraph

gr = HashGraph()
seq = ["CGA", "TTGG", "CCGT", "C", "GT", "GATAA", "CGG", "ACA", "GCCG", "ATATAAC"]
n = []
for s in seq:
    n.append(gr.create_handle(s))

gr.create_edge(n[0], n[1])
gr.create_edge(n[1], n[2])
gr.create_edge(n[2], n[3])
gr.create_edge(n[2], n[4])
gr.create_edge(n[3], n[5])
gr.create_edge(n[5], n[6])
# Connect the end of n5 to the start of n8
gr.create_edge(n[5], n[8])
gr.create_edge(n[6], n[7])
gr.create_edge(n[6], n[8])
gr.create_edge(n[7], n[9])
gr.create_edge(n[8], n[9])
# Connect the end of n8 back around to the start of n5
gr.create_edge(n[8], n[5])

def next_node_list(handle):
    lis = []
    gr.follow_edges(handle, False, lambda y: lis.append(y))
    return lis

print(f'n0: {gr.get_sequence(n[0])}')
next_node = next_node_list(n[0])[0]
print(f'n1: {gr.get_sequence(next_node)}')
next_node = next_node_list(next_node)[0]
print(f'n2: {gr.get_sequence(next_node)}')

path = gr.create_path_handle("path")
gr.append_step(path, n[0])
gr.append_step(path, n[1])
gr.append_step(path, n[2])
gr.append_step(path, n[4])
gr.append_step(path, n[5])
gr.append_step(path, n[6])
gr.append_step(path, n[7])
gr.append_step(path, n[9])

In [ ]:
from bdsg.bdsg import PackedGraph
brca2 = PackedGraph()
brca2.deserialize("/home/yboulkaid/Documents/sample_data/pgtest.data/pg.pg")
#brca2.deserialize("/home/yboulkaid/Téléchargements/cactus-brca2.pg")

path_handle = []
handles = []
brca2.for_each_path_handle(lambda y: path_handle.append(y) or True)
brca2.for_each_step_in_path(path_handle[0],
    lambda y: handles.append(brca2.get_handle_of_step(y)) or True)
sequence = ""
for handle in handles:
    sequence += brca2.get_sequence(handle)
print(sequence[0:10])
print(len(sequence))


In [ ]:
type(gr)

# CACGTCCGAGAATCGG
# CACGTCCGAG

In [ ]:
from bdsg.handlegraph import  HandleGraph
###########################################
### HOW TO ACCESS THE SNARL TREE (XIAN) ###
###########################################

#Load the distance index
distance_index = bdsg.bdsg.SnarlDistanceIndex()
distance_index.deserialize("/home/yboulkaid/Documents/sample_data/pgtest.data/pg.dist")

# net graph to build
ngraph = {'e': [], 'n': []}

# init with the child (only one ideally) of the root
root = distance_index.get_root() #Get a net handle referring to a tip-to-tip traversal of the contents of the root snarl.

def add_node_to_ngraph(net):
    ngraph['n'].append(net)
distance_index.for_each_child(root, add_node_to_ngraph)

distance_index.net_handle_as_string(ngraph['n'][0])

node3 = distance_index.get_node_net_handle(3)
parent3 = distance_index.get_parent(node3)
snarl_start_bound = distance_index.get_bound(parent3, False, True)

# for functions taking an iteratee
# print the net_handle_t and return true to continue iterating
def iterate (net) :
    print(distance_index.net_handle_as_string(net))
    return True

graph = HandleGraph()
#graph.deserialize("/home/yboulkaid/Documents/sample_data/pgtest.data/pg.pg")

print("Follow edges from", distance_index.net_handle_as_string(snarl_start_bound), ":" )
distance_index.follow_net_edges(snarl_start_bound, graph, False, iterate)


# graph: bdsg.handlegraph.HandleGraph

In [ ]:
print(ngraph)
print(distance_index.net_handle_as_string(ngraph['n'][0]))

In [ ]:
ngraph['n']

In [ ]:
distance_index.deserialize("/home/yboulkaid/Documents/sample_data/pgtest.data/pg.dist")

In [ ]:
'''get parent 
get child 
get bound 
get root'''